# Dataset
1. E3-substrates-interactions file is downloaded from : http://ubibrowser.bio-it.cn/ubibrowser_v3/home/download
2. Collected_degrons file is downloaded from: http://degron.phasep.pro/download/ 

In [1]:
import pandas as pd

In [2]:
'''Convert E3-substrates-interactions text file to Excel file'''
# read in the text file using pandas
df = pd.read_csv('../datasets/E3-substrates-interactions.txt', delimiter='\t')

# write the dataframe to an Excel file
df.to_excel('../datasets/E3-substrates-interactions.xlsx', index=False)

# Preprocessing
1. Read the datasets
2. Remove the rows with nan or invalid IDs on Entry column in degrons, and unreviewd on their Status column
3. Remove the rows with nan or invalid IDs on both SwissProt AC (Substrate) and SwissProt AC (E3) columns
4. Merge them together on Entry to a final dataset
5. Keep needed columns: SwissProt AC (Substrate), SwissProt AC (E3), protein seq, start, end, deg_seq, E3_sequence
6. Explode the rows containing # in their E3
7. Add E3 sequences to the dataframe
8. Groupby the final dataset to merge rows with the same substrate 

In [5]:
# Read in the xlsx files using pandas
degrons = pd.read_excel('../datasets/collected_degrons.xlsx')
interactions = pd.read_excel('../datasets/E3-substrates-interactions.xlsx')
E3_sequences = pd.read_excel('../datasets/uniprot-E3.xlsx')

# Remove rows from both dataframes that have NaN or invalid ids
degrons = degrons[(degrons['Entry'].notnull()) & (degrons['Entry'] != '-')]
degrons = degrons[degrons['Status'] != 'unreviewd']

# Remove the rows with nan or invalid IDs on both SwissProt AC (Substrate) and SwissProt AC (E3) columns
interactions = interactions[(interactions['SwissProt AC (Substrate)'].notnull()) & (interactions['SwissProt AC (Substrate)'] != '-')]
interactions = interactions[(interactions['SwissProt AC (E3)'].notnull()) & (interactions['SwissProt AC (E3)'] != '-')]

# Merge them together on Entry to a final dataset
final = pd.merge(degrons, interactions, left_on='Entry', right_on='SwissProt AC (Substrate)')
final = pd.merge(final, E3_sequences, left_on='SwissProt AC (E3)', right_on='Entry')

# Keep needed columns: SwissProt AC (Substrate), SwissProt AC (E3), protein seq, start, end, deg_seq
final = final[['SwissProt AC (Substrate)', 'SwissProt AC (E3)', 'protein seq', 'start', 'end', 'deg_seq', 'E3_sequence']]
final.columns = ['substrate', 'E3', 'substrate protein seq', 'start', 'end', 'degron_seq', 'E3_sequence']
final = final.assign(E3=final['E3'].str.split('#')).explode('E3')
final = final.drop_duplicates(subset=['substrate', 'E3'], keep='first')

# Creat a map to store degrons and their corresponding substrates
degron_substrate = {}
for index, row in final.iterrows():
    if row['degron_seq'] not in degron_substrate:
        degron_substrate[row['degron_seq']] = row['substrate']
        
#Save the degron_substrate dictionary to a csv file
with open('../datasets/degron_substrate.csv', 'w') as f:
    for key in degron_substrate.keys():
        f.write("%s,%s\n"%(key,degron_substrate[key]))

# Creat a dictionary to store the E3s and their corresponding E3 sequences
E3_sequence = {}
for index, row in final.iterrows():
    if row['E3'] not in E3_sequence:
        E3_sequence[row['E3']] = row['E3_sequence']

# Save the E3_sequence dictionary to a csv file
with open('../datasets/E3_sequence.csv', 'w') as f:
    for key in E3_sequence.keys():
        f.write("%s,%s\n"%(key,E3_sequence[key]))

# Groupby the final dataset to merge rows with the same substrate 
#final = final.groupby(['substrate', 'substrate protein seq', 'start', 'end', 'degron_seq']).agg({'E3': lambda x: ';'.join(x), 'E3_sequence': lambda x: ';'.join(x)}).reset_index()

# Write the final dataset to an Excel file
final.to_excel('../datasets/final_dataset.xlsx', index=False)

In [6]:
# Test and train split
from sklearn.model_selection import train_test_split
train, test = train_test_split(final, test_size=0.2, random_state=42)

# Remove substrate protein seq, start, end columns from both train and test datasets
train = train.drop(columns=['substrate protein seq', 'start', 'end'])
test = test.drop(columns=['substrate protein seq', 'start', 'end'])

#Print the size of the train and test datasets
print(train.shape)
print(test.shape)

# Write the train and test datasets to Excel files
train.to_excel('../datasets/train.xlsx', index=False)
test.to_excel('../datasets/test.xlsx', index=False)

# Create a dictionary to store the degrons and their corresponding E3s in the train dataset
# train_degron_E3 = {}
# for index, row in train.iterrows():
#     if row['degron_seq'] not in train_degron_E3:
#         train_degron_E3[row['degron_seq']] = row['E3_sequence'].split(';')

# # Save the train_degron_E3 dictionary to a csv file
# with open('../datasets/train_degron_E3.csv', 'w') as f:
#     for key in train_degron_E3.keys():
#         f.write("%s,%s\n"%(key,train_degron_E3[key]))

# # Create a dictionary to store the degrons and their corresponding E3s in the test dataset
# test_degron_E3 = {}
# for index, row in test.iterrows():
#     if row['degron_seq'] not in test_degron_E3:
#         test_degron_E3[row['degron_seq']] = row['E3_sequence'].split(';')

# # Save the test_degron_E3 dictionary to a csv file
# with open('../datasets/test_degron_E3.csv', 'w') as f:
#     for key in test_degron_E3.keys():
#         f.write("%s,%s\n"%(key,test_degron_E3[key]))



(395, 4)
(99, 4)


In [7]:
# Read train and test datasets
train = pd.read_excel('../datasets/train.xlsx')
test = pd.read_excel('../datasets/test.xlsx')

print(train.head())
print(test.head())

train_degrons = train["degron_seq"].tolist()
train_E3s = train["E3_sequence"].tolist()

test_degrons = test["degron_seq"].tolist()
test_E3s = test["E3_sequence"].tolist()

  substrate      E3      degron_seq  \
0    Q16665  P00748  LAPAAGDTIISLDF   
1    P17181  Q9Y297          DSGNYS   
2    Q16665  P10523  LAPAAGDTIISLDF   
3    P35659  Q969H0           ADSST   
4    Q13002  Q6TDP4           APVIV   

                                         E3_sequence  
0  MRALLLLGFLLVSLESTLSIPPWEAPKEHKYKAEEHTVVLTVTGEP...  
1  MDPAEAVLQEKALKFMCSMPRSLWLGCSSLADSMPSLRCLYNPGTG...  
2  MAASGKTSKSEPNHVIFKKISRDKSVTIYLGNRDYIDHVSQVQPVD...  
3  MNQELLSVGSKRRRTGGSLRGNPSSSQVDEEQMNRVVEEEQQQQLR...  
4  MQPRSERPAGRTQSPEHGSPGPGPEAPPPPPPQPPAPEAERTRPRQ...  
  substrate      E3 degron_seq  \
0    P46527  Q5XPI4   SVEQTPKK   
1    Q14247  Q75N03      ADYRE   
2    P14635  Q86WB0  PRTALGDIG   
3    O00429  Q9NX47    RKRLPVT   
4    P24864  O43791    LLTPPQS   

                                         E3_sequence  
0  MASKGAGMSFSRKSYRLTSDAEKSRVTGIVQEKLLNDYLNRIFSSS...  
1  MDHTDNELQGTNSSGSLGGLDVRRRIPIKLISKQANKAKPAPRTQR...  
2  MAAPCEGQAFAVGVEKNWGAVVRSPEGTPQKIRQLIDEGIAPEEGG...  
3  MPDQALQ

# Tokenization

In [9]:
# Reference: ProteinBERT: A Universal Deep-Learning Model of Protein Sequence and Function

# All possible amino acids
AAs = 'ACDEFGHIKLMNPQRSTUVWXY'
ADDITIONAL_TOKENS = ['<OTHER>', '<START>', '<END>', '<PAD>']

# Each sequence is added <START> and <END> tokens
ADDED_TOKENS_PER_SEQ = 2

n_aas = len(AAs)
aa_to_token_index = {aa: i for i, aa in enumerate(AAs)}
additional_token_to_index = {token: i + n_aas for i, token in enumerate(ADDITIONAL_TOKENS)}
token_to_index = {**aa_to_token_index, **additional_token_to_index}
index_to_token = {index: token for token, index in token_to_index.items()}
n_tokens = len(token_to_index)

# Tokenize the degrons and E3s in the train and test datasets
def tokenization(seq):
    other_token_index = additional_token_to_index['<OTHER>']
    return [additional_token_to_index['<START>']] + [aa_to_token_index.get(aa, other_token_index) for aa in parse_seq(seq)] + \
            [additional_token_to_index['<END>']]
            
def parse_seq(seq):
    if isinstance(seq, str):
        return seq
    elif isinstance(seq, bytes):
        return seq.decode('utf8')
    else:
        raise TypeError('Unexpected sequence type: %s' % type(seq))